Implement N-Body simulation with C code using SYLC for intelGPU

In [ ]:
Running it Sequentially First (Old Code)

In [1]:
%%writefile timer.h
#ifndef TIMER_H
#define TIMER_H

#include <stdlib.h>

#ifdef WIN32
  #define WIN32_LEAN_AND_MEAN
  #include <windows.h>
#else
  #include <sys/time.h>
   #ifndef timersub
    #define timersub(a, b, result)                          \
      do {                                                  \
        (result)->tv_sec = (a)->tv_sec - (b)->tv_sec;       \
        (result)->tv_usec = (a)->tv_usec - (b)->tv_usec;    \
        if ((result)->tv_usec < 0) {                        \
          --(result)->tv_sec;                               \
          (result)->tv_usec += 1000000;                     \
        }                                                   \
      } while (0)
  #endif
#endif

#ifdef WIN32
double PCFreq = 0.0;
__int64 timerStart = 0;
#else
struct timeval timerStart;
#endif

void StartTimer()
{
#ifdef WIN32
  LARGE_INTEGER li;
  if(!QueryPerformanceFrequency(&li))
    printf("QueryPerformanceFrequency failed!\n");

  PCFreq = (double)li.QuadPart/1000.0;

  QueryPerformanceCounter(&li);
  timerStart = li.QuadPart;
#else
  gettimeofday(&timerStart, NULL);
#endif
}

// time elapsed in ms
double GetTimer()
{
#ifdef WIN32
  LARGE_INTEGER li;
  QueryPerformanceCounter(&li);
  return (double)(li.QuadPart-timerStart)/PCFreq;
#else
  struct timeval timerStop, timerElapsed;
  gettimeofday(&timerStop, NULL);
  timersub(&timerStop, &timerStart, &timerElapsed);
    return timerElapsed.tv_sec*1000.0+timerElapsed.tv_usec/1000.0;
#endif
}

#endif // TIMER_H


Writing timer.h


In [20]:
%%writefile shmoo-cpu-nbody.sh
SRC=nbody.c
EXE=nbody
gcc -std=c99 -O3 -DSHMOO -o $EXE $SRC -lm

echo $EXE

K=1024
for i in {1..10}
do
    ./$EXE $K
    K=$(($K*2))
done

Overwriting shmoo-cpu-nbody.sh


In [25]:
%%writefile nbody.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include "timer.h"

#define SOFTENING 1e-9f

typedef struct { float x, y, z, vx, vy, vz; } Body;

void randomizeBodies(float *data, int n) {
  for (int i = 0; i < n; i++) {
    data[i] = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
  }
}

void bodyForce(Body *p, float dt, int n) {
  for (int i = 0; i < n; i++) {
    float Fx = 0.0f; 
    float Fy = 0.0f; float Fz = 0.0f;

    for (int j = 0; j < n; j++) {
      float dx = p[j].x - p[i].x;
      float dy = p[j].y - p[i].y;
      float dz = p[j].z - p[i].z;
      float distSqr = dx*dx + dy*dy + dz*dz + SOFTENING;
      float invDist = 1.0f / sqrtf(distSqr);
      float invDist3 = invDist * invDist * invDist;

      Fx += dx * invDist3; 
      Fy += dy * invDist3; 
      Fz += dz * invDist3;
    }

    p[i].vx += dt*Fx; 
    p[i].vy += dt*Fy; 
    p[i].vz += dt*Fz;
  }
}

int main(const int argc, const char** argv) {

  int nBodies = 30000;
  if (argc > 1) nBodies = atoi(argv[1]);
  printf("nBodies size is %d\n", nBodies);

  const float dt = 0.01f; // time step
  const int nIters = 10;  // simulation iterations

  int bytes = nBodies*sizeof(Body);
  float *buf = (float*)malloc(bytes);
  Body *p = (Body*)buf;

  randomizeBodies(buf, 6*nBodies); // Init pos / vel data

  double totalTime = 0.0;

  for (int iter = 1; iter <= nIters; iter++) {
    StartTimer();

    bodyForce(p, dt, nBodies); // compute interbody forces

    for (int i = 0 ; i < nBodies; i++) { // integrate position
      p[i].x += p[i].vx*dt;
      p[i].y += p[i].vy*dt;
      p[i].z += p[i].vz*dt;
    }

    const double tElapsed = GetTimer() / 1000.0;
    if (iter > 1) { // First iter is warm up
      totalTime += tElapsed;
    }
#ifndef DSHMOO
    printf("Iteration %d: %.3f seconds\n", iter, tElapsed);
#endif
  }
  double avgTime = totalTime / (double)(nIters-1);
  printf( "Average time: %.3f seconds\n", avgTime);


#ifdef DSHMOO
  printf("%d Bodies: average %0.3f Billion Interactions / second\n", nBodies, 1e-9 * nBodies * nBodies / avgTime);
#endif

  free(buf);
}

Overwriting nbody.c


In [2]:
! gcc -std=c99 -O3 -DSHMOO timer.h -o nbody nbody.c -lm
! chmod 777 shmoo-cpu-nbody.sh
!./shmoo-cpu-nbody.sh

nbody
nBodies size is 1024
Iteration 1: 0.006 seconds
Iteration 2: 0.005 seconds
Iteration 3: 0.005 seconds
Iteration 4: 0.006 seconds
Iteration 5: 0.005 seconds
Iteration 6: 0.005 seconds
Iteration 7: 0.005 seconds
Iteration 8: 0.005 seconds
Iteration 9: 0.005 seconds
Iteration 10: 0.005 seconds
Average time: 0.005 seconds
nBodies size is 2048
Iteration 1: 0.025 seconds
Iteration 2: 0.024 seconds
Iteration 3: 0.021 seconds
Iteration 4: 0.017 seconds
Iteration 5: 0.015 seconds
Iteration 6: 0.015 seconds
Iteration 7: 0.015 seconds
Iteration 8: 0.015 seconds
Iteration 9: 0.015 seconds
Iteration 10: 0.015 seconds
Average time: 0.017 seconds
nBodies size is 4096
Iteration 1: 0.106 seconds
Iteration 2: 0.063 seconds
Iteration 3: 0.063 seconds
Iteration 4: 0.063 seconds
Iteration 5: 0.063 seconds
Iteration 6: 0.063 seconds
Iteration 7: 0.063 seconds
Iteration 8: 0.063 seconds
Iteration 9: 0.063 seconds
Iteration 10: 0.063 seconds
Average time: 0.063 seconds
nBodies size is 8192
Iteration 1: 

Running it with SYLC

In [18]:
%%writefile shmoo-gpu-nbody.sh
SRC=nbody.cpp
EXE=nbody
icpx -fsycl -std=c++17 -O3 -DSHMOO -o $EXE $SRC -lm

echo $EXE

K=1024
for i in {1..10}
do
    ./$EXE $K
    K=$(($K*2))
done

Overwriting shmoo-gpu-nbody.sh


In [16]:
%%writefile nbody.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <cmath>
#include <cstdlib>
#include "timer.h"

#define SOFTENING 1e-9f

struct Body {
    float x, y, z, vx, vy, vz;
};

// Generate random values on the CPU
void randomizeBodies(float *data, int n) {
    for (int i = 0; i < n; i++) {
        data[i] = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
    }
}

void bodyForce(Body *p, float dt, int n, sycl::queue &q) {
    q.submit([&](sycl::handler &h) {
        h.parallel_for(sycl::range<1>(n), [=](sycl::id<1> i) {
            int idx = i[0];
            float Fx = 0.0f;
            float Fy = 0.0f;
            float Fz = 0.0f;

            for (int j = 0; j < n; j++) {
                float dx = p[j].x - p[idx].x;
                float dy = p[j].y - p[idx].y;
                float dz = p[j].z - p[idx].z;
                float distSqr = dx * dx + dy * dy + dz * dz + SOFTENING;
                float invDist = 1.0f / sycl::sqrt(distSqr);
                float invDist3 = invDist * invDist * invDist;

                Fx += dx * invDist3;
                Fy += dy * invDist3;
                Fz += dz * invDist3;
            }

            p[idx].vx += dt * Fx;
            p[idx].vy += dt * Fy;
            p[idx].vz += dt * Fz;
        });
    }).wait();
}

void integrate(Body *p, float dt, int n, sycl::queue &q) {
    q.submit([&](sycl::handler &h) {
        h.parallel_for(sycl::range<1>(n), [=](sycl::id<1> i) {
            int idx = i[0];
            p[idx].x += p[idx].vx * dt;
            p[idx].y += p[idx].vy * dt;
            p[idx].z += p[idx].vz * dt;
        });
    }).wait();
}

int main(int argc, char** argv) {
    int nBodies = 30000;
    if (argc > 1) nBodies = atoi(argv[1]);
   std::cout << "nBodies is " << nBodies << std::flush;

    const float dt = 0.01f; // time step
    const int nIters = 10;  // simulation iterations

    // Allocate memory for bodies on the CPU
    Body *bodiesCPU = new Body[nBodies];

    // Generate random values on the CPU
    randomizeBodies(reinterpret_cast<float*>(bodiesCPU), 6 * nBodies); // Init pos / vel data

    // Create a SYCL queue
    sycl::queue q(sycl::gpu_selector_v);

    // Allocate shared memory for bodies on the GPU
    Body *bodiesGPU = sycl::malloc_shared<Body>(nBodies, q);

    // Copy bodies from CPU to GPU
    q.memcpy(bodiesGPU, bodiesCPU, nBodies * sizeof(Body)).wait();

    double totalTime = 0.0;

    for (int iter = 1; iter <= nIters; iter++) {
        StartTimer();

        bodyForce(bodiesGPU, dt, nBodies, q); // compute interbody forces
        integrate(bodiesGPU, dt, nBodies, q); // integrate

        const double tElapsed = GetTimer() / 1000.0;
        if (iter > 1) { // First iter is warm up
            totalTime += tElapsed;
        }
#ifndef DSHMOO
        std::cout << "Iteration " << iter << ": " << tElapsed << " seconds\n" << std::flush;
#endif
    }
    double avgTime = totalTime / (double)(nIters - 1);
    std::cout << "Average time: " << avgTime << " seconds\n" << std::flush;

#ifdef DSHMOO
    std::cout << nBodies << " Bodies: average " << 1e-9 * nBodies * nBodies / avgTime << " Billion Interactions / second\n" << std::flush;
#endif

    // Free memory
    sycl::free(bodiesGPU, q);
    delete[] bodiesCPU;
    return 0;
}

Overwriting nbody.cpp


In [19]:
! icpx -fsycl -std=c++17 -O3 -DSHMOO -o nbody nbody.cpp -lm
! chmod 777 shmoo-gpu-nbody.sh
! ./shmoo-gpu-nbody.sh

nbody
nBodies is 1024
Iteration 1: 0.126547 seconds
Iteration 2: 0.000172 seconds
Iteration 3: 0.000166 seconds
Iteration 4: 0.000164 seconds
Iteration 5: 0.000157 seconds
Iteration 6: 0.000158 seconds
Iteration 7: 0.000157 seconds
Iteration 8: 0.000159 seconds
Iteration 9: 0.000158 seconds
Iteration 10: 0.000158 seconds
Average time: 0.000161 seconds
nBodies is 2048
Iteration 1: 0.121162 seconds
Iteration 2: 0.000399 seconds
Iteration 3: 0.000397 seconds
Iteration 4: 0.000386 seconds
Iteration 5: 0.000391 seconds
Iteration 6: 0.000387 seconds
Iteration 7: 0.000391 seconds
Iteration 8: 0.000386 seconds
Iteration 9: 0.000391 seconds
Iteration 10: 0.000386 seconds
Average time: 0.000390444 seconds
nBodies is 4096
Iteration 1: 0.121476 seconds
Iteration 2: 0.000878 seconds
Iteration 3: 0.000868 seconds
Iteration 4: 0.000868 seconds
Iteration 5: 0.000869 seconds
Iteration 6: 0.000867 seconds
Iteration 7: 0.000867 seconds
Iteration 8: 0.000866 seconds
Iteration 9: 0.000869 seconds
Iteration

Discussion: We can see here how much the introduction of SYLC has allowed for speed-up from the 
old sequential code. Previously, when we ran the code with nBodies of 524K, with sequential the 
results were 605 seconds but with the intelGPU the average time per iteration is now only 0.8 seconds. 

The biggest difference in the code is that I offloaded the data from the CPU into the GPU 
and started parallelizing the bodyForce and integrate functions. This decision allows us to
take advantage of the GPU's many many cores. The GPU functions now run the for loop operations
in parallel not having to wait for any previous for loop operation - causing very little delay. 
We create a SYCL queue that we can submit actions/code so that the queue can schedule and execute it.
This will allow our kernels to be executed in parallel.